### Installation

In [1]:
!pip install unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
  
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2025.3.17 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
gprmax_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = gprmax_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("IraGia/gprMax_Train", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

data.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

data2.json:   0%|          | 0.00/3.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/12000 [00:00<?, ? examples/s]

In [6]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
5.516 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.029600
2,2.117500
3,1.929700
4,1.901200
5,2.012000
6,1.848700
7,1.732300
8,1.849400
9,1.609900
10,1.526500


In [8]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1732.5249 seconds used for training.
28.88 minutes used for training.
Peak reserved memory = 9.305 GB.
Peak reserved memory for training = 3.789 GB.
Peak reserved memory % of max memory = 41.975 %.
Peak reserved memory for training % of max memory = 17.092 %.


In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    gprmax_prompt.format(
        "write this input file", # instruction
        "The receiver should be placed at 4.5696, 58.6801, 36.3701 The internal resistance of the voltage source is 831.6273451640435 Ohms The discretisation step should be 1.5901 meters. I want the PML to be 9 cell thick The dimensions of the model are 15.8559 x 75.1634 x 39.5206 meters. The central frequency will be equal with 0.134 GHz Generate random layers The duration of the signal will be 2.2815719340168907e-05 seconds The polarization of the pulse should 'x' The pulse should be gaussianprime Tx is a Hertzian Dipole", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nwrite this input file\n\n### Input:\nThe receiver should be placed at 4.5696, 58.6801, 36.3701 The internal resistance of the voltage source is 831.6273451640435 Ohms The discretisation step should be 1.5901 meters. I want the PML to be 9 cell thick The dimensions of the model are 15.8559 x 75.1634 x 39.5206 meters. The central frequency will be equal with 0.134 GHz Generate random layers The duration of the signal will be 2.2815719340168907e-05 seconds The polarization of the pulse should 'x' The pulse should be gaussianprime Tx is a Hertzian Dipole\n\n### Response:\n#domain: 15.8559 75.1634 39.5206 \n#dx_dy_dz: 1.5901 1.5901 1.5901 \n#time_window: 2.2815719340168907e-05 \n#waveform: gaussianprime"]

In [10]:
# gprmax_prompt
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    gprmax_prompt.format(
        "Make this model", # instruction
        "Generate random layers The dimensions of the model are 2.3594 x 41.214 x 66.2451 meters. The excitation pulse should be gaussiandotdotnorm Tx is a Hertzian Dipole I want the PML to be 77 cell thick The duration of the signal will be 1.1431824134811208e-05 seconds The central frequency will be equal with 3877.271 MHz The source should be placed at the coordinates 0.2455, 20.9791, 49.2353 meters The internal resistance of the voltage source is 630.1484646516255 Ohms The receiver should be placed at the coordinates (0.2455, 20.9791, 49.2353) meters The polarization of the pulse should 'x'", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Make this model

### Input:
Generate random layers The dimensions of the model are 2.3594 x 41.214 x 66.2451 meters. The excitation pulse should be gaussiandotdotnorm Tx is a Hertzian Dipole I want the PML to be 77 cell thick The duration of the signal will be 1.1431824134811208e-05 seconds The central frequency will be equal with 3877.271 MHz The source should be placed at the coordinates 0.2455, 20.9791, 49.2353 meters The internal resistance of the voltage source is 630.1484646516255 Ohms The receiver should be placed at the coordinates (0.2455, 20.9791, 49.2353) meters The polarization of the pulse should 'x'

### Response:
#domain: 2.3594 41.214 66.2451 
#dx_dy_dz: 2.3594 2.3594 2.3594 
#time_window: 1.1431824134811208e-05 
#waveform: gaussiandotdotnorm 1 3877.271e6 my_pulse 
#hertzian_dipol

In [11]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [12]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# gprmax_prompt = You MUST copy from above!

inputs = tokenizer(
[
    gprmax_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
The Eiffel Tower is a famous tall tower in Paris. 

Please let me know if you want me to make any changes.<|eot_id|>


In [18]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 33.51 out of 60.46 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 16/32 [00:00<00:00, 18.78it/s]

100%|██████████| 32/32 [00:02<00:00, 14.90it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file ('llama.cpp/llama-quantize' or 'llama.cpp/llama-quantize.exe' if you are on Windows WSL) or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name or check extension of the llama-quantize file.

In [1]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


⚡️ Tip	Check organization access: https://github.com/settings/connections/applications/c7457225b242a94d60c6



In [2]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/ggml-kompute/kompute'
Cloning into '/teamspace/studios/this_studio/llama.cpp/ggml/src/ggml-kompute/kompute'...
Submodule path 'ggml/src/ggml-kompute/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'



In [3]:
!make clean -C llama.cpp

make: Entering directory '/teamspace/studios/this_studio/llama.cpp'
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
make: Leaving directory '/teamspace/studios/this_studio/llama.cpp'


In [4]:
!sudo apt install cmake -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [6]:
!cmake --version


cmake version 3.16.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).
